### Convirtiendo los archivos en Json para luego subir a Azure Cognitive Search

Bajando las bibliotecas:
* PyPDF2
* python-docx
* openpyxl

In [1]:
! pip install PyPDF2 python-docx openpyxl pandas xlrd python-dotenv



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import os
import json
from PyPDF2 import PdfReader
from docx import Document  # Para procesar archivos Word
import openpyxl  # Para procesar archivos Excel
import warnings
from openpyxl import Workbook
import xlrd

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Ruta del directorio donde están los documentos
input_dir = os.getenv("INPUT_DIR")
output_json = os.getenv("OUTPUT_JSON")

def convertir_xls_a_xlsx(ruta_xls, ruta_xlsx):
    libro_xls = xlrd.open_workbook(ruta_xls)
    libro_xlsx = Workbook()
    hoja_xlsx = libro_xlsx.active

    hoja_xls = libro_xls.sheet_by_index(0)
    for row in range(hoja_xls.nrows):
        for col in range(hoja_xls.ncols):
            hoja_xlsx.cell(row=row+1, col=col+1).value = hoja_xls.cell_value(row, col)

    libro_xlsx.save(ruta_xlsx)

def extraer_texto_archivo(file_path, file_type):
    try:
        if file_type == "pdf":
            reader = PdfReader(file_path)
            text = " ".join([page.extract_text() for page in reader.pages])
        elif file_type == "docx":
            doc = Document(file_path)
            text = "\n".join([p.text for p in doc.paragraphs])
        elif file_type in ["xlsx", "xls"]:
            wb = openpyxl.load_workbook(file_path, data_only=True)
            text = ""
            for sheet in wb.sheetnames:
                ws = wb[sheet]
                for row in ws.iter_rows(values_only=True):
                    text += " ".join([str(cell) for cell in row if cell is not None]) + "\n"
            wb.close()
        else:
            text = None
        if text:
            text = text.encode("utf-8", "replace").decode("utf-8")
        return text
    except Exception as e:
        print(f"Error procesando {file_path}: {e}")
        return None

def convertir_a_json_recursivamente(directorio):
    documentos = []
    for root, _, files in os.walk(directorio):
        for filename in files:
            if filename.startswith(".") or filename.endswith((".zip", ".DS_Store")):
                continue
            file_path = os.path.join(root, filename)
            if filename.endswith(".pdf"):
                text = extraer_texto_archivo(file_path, "pdf")
            elif filename.endswith(".docx"):
                text = extraer_texto_archivo(file_path, "docx")
            elif filename.endswith(".xlsx"):
                text = extraer_texto_archivo(file_path, "xlsx")
            elif filename.endswith(".xls"):
                text = extraer_texto_archivo(file_path, "xls")
            else:
                print(f"Tipo de archivo no soportado: {filename}")
                continue

            if text:
                documento = {
                    "id": filename,
                    "path": file_path,
                    "content": text
                }
                documentos.append(documento)

    try:
        with open(output_json, "w", encoding="utf-8") as f:
            json.dump(documentos, f, ensure_ascii=False, indent=4)
        print(f"Documentos convertidos a JSON y guardados en {output_json}")
    except UnicodeEncodeError as e:
        print(f"Error al guardar JSON: {e}")

# Llamar a la función con la variable input_dir
convertir_a_json_recursivamente(input_dir)


FloatObject (b'0.00-10') invalid; use 0.0 instead
FloatObject (b'0.00-10') invalid; use 0.0 instead
FloatObject (b'0.00-10') invalid; use 0.0 instead
FloatObject (b'0.00-10') invalid; use 0.0 instead
FloatObject (b'0.00-10') invalid; use 0.0 instead


Documentos convertidos a JSON y guardados en /Users/luisalbertocerelli/Desktop/00-Todo/14_Hackaton_Real/PRINCIPAL/01Conversion_File_Json/Json_conversion/documentos_indexados.json
